In [13]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Test Model

### Get Test Data and Model

In [3]:
# MAKE SURE THIS IS UP TO DATE WITH THE CLASS DEFINED IN model_training.ipynb

class RelevanceDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = torch.tensor(X_data.reset_index(drop=True).embeddings)
        self.y_data = torch.tensor(y_data.reset_index(drop=True).relevance).long()

    def __getitem__(self, item):
        return self.X_data[item], self.y_data[item]

    def __len__(self):
        return len(self.X_data)

In [4]:
test_dataset = torch.load('data/test_dataset.pt')
test_loader = DataLoader(dataset=test_dataset, batch_size=1)
model = torch.load('models/model_v1.pt')

#### Conduct Test

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
y_pred_list = []

with torch.no_grad():
    model.eval()

    for X_batch, _, in test_loader:
        X_batch = X_batch.to(device)

        y_test_pred = model(X_batch)

        _, y_pred_tags = torch.max(y_test_pred, dim=1)
        y_pred_list.append((y_pred_tags.cpu().numpy()))

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

#### Confusion Matrix and Classifcation Report

In [12]:
y_test = [d[1].item() for d in test_dataset]
y_test

[0, 0, 0, 0, 0, 2, 3, 3, 0, 2, 0, 1, 1, 3, 3, 0, 1, 2, 3, 0]

In [ ]:
cm_df = pd.DataFrame(confusion_matrix(y_test, y_pred_list))

sns.heatmap(cm_df, annot=True)

In [ ]:
classification_report(y_test, y_pred_list)